Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

### P3. Para el promedio de stars por cada ciudad, ¿cuáles son los 5 promedios más comunes?

In [3]:
import pandas as pd

reviews = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/review.csv', usecols=['business_id', 'stars'])
reviews

,business_id,stars
0,XQfwVwDr-v0ZS3_CbbE5Xw,3.0
1,e4Vwtrqf-wpJfwesgvdgxQ,4.0
2,cPepkJeRMtHapc_b2Oe_dw,4.0
3,bMratNjTG5ZFEA6hVyr-xQ,5.0
4,bbEXAEFr4RYHLlZ-HFssTA,5.0
...,...,...
3838100,6WaI-IN8ql0xpEKlb4q8tg,5.0
3838101,2vLksaMmSEcGbjI5gywpZA,5.0
3838102,R1khUUxidqfaJmcpmGd4aw,4.0
3838103,Rr9kKArrMhSLVE9a53q-aA,5.0


In [4]:
businesses = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/business.csv', usecols=['business_id', 'city'])
businesses

,business_id,city
0,Pns2l4eNsfO8kk83dixA6A,Santa Barbara
1,mpf3x-BjTdTEA3yCZrAYPw,Affton
2,tUFrWirKiKi_TAnsVWINQQ,Tucson
3,MTSW4McQd7CbVtyjqoe9mw,Philadelphia
4,mWMc6_wTdE0EUBKIGXDVfA,Green Lane
...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Edmonton
150342,c8GjPIOTGVmIemT7j5_SyQ,Nashville
150343,_QAMST-NrQobXduilWEqSw,Indianapolis
150344,mtGm22y5c2UHNXDFAjaPNw,Edwardsville


Mergeo los dataframes usando la columna business_id que es común a ambos

In [5]:
cities_reviews = pd.merge(businesses, reviews)
cities_reviews

,business_id,city,stars
0,mpf3x-BjTdTEA3yCZrAYPw,Affton,5.0
1,mpf3x-BjTdTEA3yCZrAYPw,Affton,5.0
2,mpf3x-BjTdTEA3yCZrAYPw,Affton,1.0
3,mpf3x-BjTdTEA3yCZrAYPw,Affton,4.0
4,mpf3x-BjTdTEA3yCZrAYPw,Affton,1.0
...,...,...,...
3838100,jV_XOycEzSlTx-65W906pg,Apollo beach,5.0
3838101,jV_XOycEzSlTx-65W906pg,Apollo beach,5.0
3838102,jV_XOycEzSlTx-65W906pg,Apollo beach,5.0
3838103,jV_XOycEzSlTx-65W906pg,Apollo beach,5.0


Obtengo los 5 promedios más comunes:

In [6]:
cities_reviews.groupby("city").mean().value_counts().head(5)

stars
5.0      82
1.0      27
4.0      25
3.0      19
2.0      13
dtype: int64

Se observa que los 5 promedios más comunes por ciudad son 5.0, 1.0, 4.0, 3.0 y 2.0. Esto ocurre ya que muchas ciudades tienen una sola review, o pocas reviews que se promedian a una cantidad entera, mientras que para ciudades con muchas reviews es poco probable que sus promedios coincidan exactamente.  

### P25. Yelp quiere reconocer qué usuarios están progresando en el sitio de manera excepcional respecto a la mayoría para darles una medallita. El problema es que todos los usuarios tienen distintas edades en el sitio y eso dificulta medir progreso. Calcule la velocidad con que cada usuario consiguió fans e hizo reviews en el sitio. ¿Cuántos usuarios superan ambas velocidades promedio?

In [18]:
import pandas as pd

users = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/user.csv', usecols=['review_count', 'fans', 'yelping_since'])
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   review_count   float64
 1   yelping_since  object 
 2   fans           int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 45.5+ MB


In [19]:
users['yelping_since'] = pd.to_datetime(users['yelping_since'])
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   review_count   float64       
 1   yelping_since  datetime64[ns]
 2   fans           int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 45.5 MB


Calculo los días que lleva cada usuario en la plataforma:

In [20]:
TODAY = pd.to_datetime("2022.05.02")

def days(date):
  return (TODAY - date).days

users["days_old"] = users["yelping_since"].map(days)
users

,review_count,yelping_since,fans,days_old
0,5.0,2007-01-25 16:47:26,267,5575
1,64.0,2009-01-25 04:35:42,3138,4844
2,0.0,2008-07-25 10:41:00,52,5028
3,0.0,2005-11-29 04:38:33,28,5997
4,7.0,2007-01-05 19:40:59,1,5595
...,...,...,...,...
1987892,1.0,2015-01-06 00:31:31,0,2672
1987893,1.0,2016-06-14 07:20:52,0,2147
1987894,1.0,2017-02-04 15:31:58,0,1912
1987895,1.0,2011-01-14 00:29:08,0,4125


Calculo la velocidad de progreso con la que cada usuario realizó reviews y consiguió fans:

In [21]:
users["review_progress"] = users["review_count"] / users["days_old"]
users["fans_progress"] = users["fans"] / users["days_old"]
users

,review_count,yelping_since,fans,days_old,review_progress,fans_progress
0,5.0,2007-01-25 16:47:26,267,5575,0.000897,0.047892
1,64.0,2009-01-25 04:35:42,3138,4844,0.013212,0.647812
2,0.0,2008-07-25 10:41:00,52,5028,0.000000,0.010342
3,0.0,2005-11-29 04:38:33,28,5997,0.000000,0.004669
4,7.0,2007-01-05 19:40:59,1,5595,0.001251,0.000179
...,...,...,...,...,...,...
1987892,1.0,2015-01-06 00:31:31,0,2672,0.000374,0.000000
1987893,1.0,2016-06-14 07:20:52,0,2147,0.000466,0.000000
1987894,1.0,2017-02-04 15:31:58,0,1912,0.000523,0.000000
1987895,1.0,2011-01-14 00:29:08,0,4125,0.000242,0.000000


Para cada usuario se verifica si las velocidades con las que realizó reviews y consiguió fans superan a las promedio, y si supera ambas se lo reconocerá como usuario excepcional.

In [22]:
review_mean, fan_mean = users.agg({"review_progress":"mean", "fans_progress":"mean"})
progress = users.agg({"review_progress": lambda x: x > review_mean, "fans_progress": lambda y: y > fan_mean})
progress["exceptional_user"] = (progress["review_progress"] & progress["fans_progress"])
progress

,review_progress,fans_progress,exceptional_user
0,False,True,False
1,True,True,True
2,False,True,False
3,False,True,False
4,True,False,False
...,...,...,...
1987892,False,False,False
1987893,False,False,False
1987894,False,False,False
1987895,False,False,False


In [23]:
progress["exceptional_user"].sum()

68761

Por lo que hay 68761 usuarios que merecen el reconocimiento al superan el progreso promedio de fans y reviews.